In [1]:
import rootutils

rootutils.setup_root("/TopoBenchmarkX", indicator=".project-root", pythonpath=True)

from hydra.experimental import initialize, compose

initialize(config_path="../configs", job_name="job")
config = compose(config_name="train.yaml")

/usr/local/lib/python3.11/site-packages/hydra/experimental/initialize.py:43: UserWarning: hydra.experimental.initialize() is no longer experimental. Use hydra.initialize()
  deprecation_warning(message=message)
/usr/local/lib/python3.11/site-packages/hydra/experimental/initialize.py:45: UserWarning: 
The version_base parameter is not specified.
Please specify a compatability version level, or None.
Will assume defaults for version 1.1
  self.delegate = real_initialize(
/usr/local/lib/python3.11/site-packages/hydra/experimental/compose.py:25: UserWarning: hydra.experimental.compose() is no longer experimental. Use hydra.compose()
  deprecation_warning(message=message)


In [3]:
config["data"]

'cora'

In [14]:
import torch
import pickle
import numpy as np

import os.path as osp

from torch_geometric.data import Data
from torch_sparse import coalesce


def load_citation_dataset(cfg):
    """
    this will read the citation dataset from HyperGCN, and convert it edge_list to
    [[ -V- | -E- ]
     [ -E- | -V- ]]
    """
    data_dir = cfg["data"]["data_dir"]
    print(
        f"Loading {cfg['data']['data_domain']} dataset name: {cfg['data']['data_name']}"
    )

    # first load node features:

    with open(osp.join(data_dir, "features.pickle"), "rb") as f:
        features = pickle.load(f)
        features = features.todense()

    # then load node labels:
    with open(osp.join(data_dir, "labels.pickle"), "rb") as f:
        labels = pickle.load(f)

    num_nodes, feature_dim = features.shape
    assert num_nodes == len(labels)
    print(f"number of nodes:{num_nodes}, feature dimension: {feature_dim}")

    features = torch.FloatTensor(features)
    labels = torch.LongTensor(labels)

    # The last, load hypergraph.
    with open(osp.join(data_dir, "hypergraph.pickle"), "rb") as f:
        # hypergraph in hyperGCN is in the form of a dictionary.
        # { hyperedge: [list of nodes in the he], ...}
        hypergraph = pickle.load(f)

    print(f"number of hyperedges: {len(hypergraph)}")

    edge_idx = num_nodes
    node_list = []
    edge_list = []
    for he in hypergraph.keys():
        cur_he = hypergraph[he]
        cur_size = len(cur_he)

        node_list += list(cur_he)
        edge_list += [edge_idx] * cur_size

        edge_idx += 1

    edge_index = np.array([node_list + edge_list, edge_list + node_list], dtype=int)
    edge_index = torch.LongTensor(edge_index)

    data = Data(x=features, edge_index=edge_index, y=labels)

    # data.coalesce()
    # There might be errors if edge_index.max() != num_nodes.
    # used user function to override the default function.
    # the following will also sort the edge_index and remove duplicates.
    total_num_node_id_he_id = edge_index.max() + 1
    data.edge_index, data.edge_attr = coalesce(
        data.edge_index, None, total_num_node_id_he_id, total_num_node_id_he_id
    )

    n_x = num_nodes
    #     n_x = n_expanded
    num_class = len(np.unique(labels.numpy()))
    # val_lb = int(n_x * train_percent)
    # percls_trn = int(round(train_percent * n_x / num_class))
    # data = random_planetoid_splits(data, num_class, percls_trn, val_lb)
    data.n_x = n_x
    # add parameters to attribute

    # data.train_percent = train_percent
    data.num_hyperedges = len(hypergraph)

    return data

In [15]:
load_citation_dataset(config)

Loading hypergraph dataset name: cora
number of nodes:2708, feature dimension: 1433
number of hyperedges: 1072


/tmp/ipykernel_41034/2768844153.py:22: DeprecationWarning: Please use `csr_matrix` from the `scipy.sparse` namespace, the `scipy.sparse.csr` namespace is deprecated.
  features = pickle.load(f)


Data(x=[2708, 1433], edge_index=[2, 9170], y=[2708], n_x=2708, num_hyperedges=1072)

'hypergraph'